<a href="https://colab.research.google.com/github/DheerajKP1/15-puzzle/blob/master/project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install torch
!pip install simpletransformers
!pip install sentencepiece




In [ ]:
import numpy as np
import pandas as pd
import torch
from transformers import (
    CamembertTokenizer,
    AutoTokenizer,
    AutoModel,
    RobertaModel,
    AutoModelForMaskedLM,
    AutoModelForSequenceClassification,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    pipeline,
)

In [ ]:
df = pd.read_excel("/content/train of malay_hate_sentiment_dataset.xlsx")


In [ ]:
df

,Tweets,Sentiment,Unnamed: 2
0,"bela ayam dlm apartment, wow d",Neutral,NaN
1,"kau kalau nak buat kerja lambat lambat, jangan...",Negative,NaN
2,"okay kita try abaya , asyik takde je abaya col...",Neutral,NaN
3,farteees janji janji manis kontraktor nyampah ...,Positive,NaN
4,geng vietnam memang suka makan kucing dengan a...,Negative,NaN
...,...,...,...
2196,ayang utriza khofifahip untagsurabaya kejaksaa...,Negative,NaN
2197,pelacur yahudi yg tolong anjing pun dpt masuk ...,Negative,NaN
2198,time tuh mmg rasa lifeless gila sbb gaji mmg c...,Positive,NaN
2199,anjing pun dia bedal puak viet nie,Negative,NaN


In [ ]:
df=df.drop('Unnamed: 2',axis=1)


In [ ]:
df

,Tweets,Sentiment
0,"bela ayam dlm apartment, wow d",Neutral
1,"kau kalau nak buat kerja lambat lambat, jangan...",Negative
2,"okay kita try abaya , asyik takde je abaya col...",Neutral
3,farteees janji janji manis kontraktor nyampah ...,Positive
4,geng vietnam memang suka makan kucing dengan a...,Negative
...,...,...
2196,ayang utriza khofifahip untagsurabaya kejaksaa...,Negative
2197,pelacur yahudi yg tolong anjing pun dpt masuk ...,Negative
2198,time tuh mmg rasa lifeless gila sbb gaji mmg c...,Positive
2199,anjing pun dia bedal puak viet nie,Negative


In [ ]:

df=df.dropna(subset=['Tweets'])

In [ ]:
df.describe()

,Tweets,Sentiment
count,2201,2201
unique,2156,4
top,anjing lain yg gigit kau anjing lain yg kene t...,Negative
freq,3,1620


In [ ]:
#print(df.iloc[4887])

In [ ]:
public_models = ['xlm-roberta-base', 'bert-base-multilingual-cased']
model_name = "wangchanberta-base-att-spm-uncased"
tokenizers = {
    'wangchanberta-base-att-spm-uncased': AutoTokenizer,
    'bert-base-multilingual-cased' : AutoTokenizer
}

In [ ]:
tokenizer = tokenizers[public_models[1]].from_pretrained(
                'bert-base-multilingual-cased',force_download=True
               )


In [ ]:
from transformers import XLNetModel, XLNetTokenizer
xlnet_model = 'malay-huggingface/xlnet-large-bahasa-cased'

In [ ]:
model = XLNetModel.from_pretrained('malay-huggingface/xlnet-large-bahasa-cased',output_hidden_states=True)
tokenizer = XLNetTokenizer.from_pretrained(xlnet_model
)


In [ ]:
model=RobertaModel.from_pretrained('bert-base-multilingual-cased',output_hidden_states=True,force_download=True)
# Put the model in "evaluation" mode, meaning feed-forward operation

You are using a model of type bert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.


Some weights of RobertaModel were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['encoder.layer.8.attention.self.query.bias', 'encoder.layer.9.output.dense.bias', 'encoder.layer.7.attention.self.key.bias', 'encoder.layer.6.attention.output.dense.bias', 'encoder.layer.3.output.LayerNorm.bias', 'encoder.layer.3.attention.self.key.bias', 'encoder.layer.6.output.LayerNorm.bias', 'encoder.layer.11.attention.self.value.weight', 'encoder.layer.2.attention.self.value.bias', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.3.attention.self.value.weight', 'encoder.layer.7.output.dense.weight', 'encoder.layer.6.attention.self.key.bias', 'encoder.layer.7.output.LayerNorm.bias', 'encoder.layer.11.attention.output.LayerNorm.weight', 'encoder.layer.6.attention.output.LayerNorm.weight', 'encoder.layer.5.attention.output.dense.bias', 'encoder.layer.7.attention.self.value.bias', 'encoder.layer.1.atte

In [ ]:
print(df.iloc[2200])

Tweets       mati itu pasti tco oxaifm qlq
Sentiment                         Negative
Name: 2200, dtype: object


In [ ]:
import re

In [ ]:
model.eval()

final_word_embedding=[]
def get_sentencewise_embed():
  for it in range(len(df)):
    if it%100 == 0:
      print(it)

    text = df.loc[it,'Tweets']
    if len(text)==0 :
      continue
    text=text.lower()

    text = text.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')
    text = text.replace(r"[\"\'\|\?\=\.\@\#\*\,\\n]+", '')
    text = text.replace("�", "")
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    print(text)
    # Add the special tokens.
    marked_text = "[CLS] " + str(text) + " [SEP]"
    # Split the sentence into tokens.
    tokenized_text = tokenizer.tokenize(marked_text)
    if len(tokenized_text)>=509:
      tokenized_text=tokenized_text[:506]
      tokenized_text=tokenized_text+[ '▁[', 'SEP', ']']
      print(len(tokenized_text))
    # Map the token strings to their vocabulary indeces.
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    encoded_layers = model(tokens_tensor, segments_tensors)[2]
    token_embeddings = torch.stack(encoded_layers, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1,0,2)
    print(len(token_embeddings))
    token_vecs_sum = []
    # For each token in the sentence...
    for token in token_embeddings:
      sum_vec = torch.sum(token[-4:], dim=0)
      print(sum_vec)
      sum_vec=sum_vec.detach().numpy()
      token_vecs_sum.append(sum_vec)
    final_word_embedding.append(token_vecs_sum)


In [ ]:
get_sentencewise_embed()
print(len(final_word_embedding))
print(len(final_word_embedding[0]))
print(len(final_word_embedding[0][0]))
#final_word_embedding=final_word_embedding.detach().numpy()
#np.save('Malay_word_embedding',Word_Embeddings)

Streaming output truncated to the last 5000 lines.
         5.2729e+00,  7.9316e+00, -5.6718e+00, -3.9548e+00,  5.5188e+00,
         1.4842e+00,  4.4543e+00,  5.6177e-01,  2.4295e+00,  1.7029e+00,
         2.1649e+00,  5.8037e+00,  2.4411e-01, -1.1985e+00, -2.2957e+00,
         7.3590e+00,  1.8008e+00,  2.5299e+00, -2.8983e+00, -4.5928e+00,
        -5.1665e+00,  3.6606e+00,  1.6583e+00,  4.7909e+00,  1.3443e+00,
         2.5829e+00, -3.8646e+00, -2.7936e+00,  2.2354e+00, -4.5789e-01,
         3.4497e+00, -1.7937e+00,  2.1241e+00,  2.4915e+00,  3.9457e+00,
        -2.1971e+00,  1.4650e+00,  5.3274e+00, -5.5634e-02,  8.6539e-01,
        -2.3623e+00,  3.6305e+00,  2.3764e+00,  4.2096e+00,  5.3281e-01,
         1.5352e-01, -5.9879e+00,  3.5185e+00,  3.2046e+00,  9.1696e-01,
        -3.4199e+00, -9.3638e-01, -2.0907e+00, -7.9432e+00, -6.9513e-01,
         3.8722e+00,  1.6579e+00,  5.4078e+00, -5.1411e+00,  3.7864e-01,
        -8.8313e+00, -7.1242e-01,  6.8227e-01, -5.0994e+00,  3.2778e+00,


In [ ]:
df['X_train_embedded'] = np.array(final_word_embedding)
df['X_train_embedded']

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

In [ ]:
df['Sentiment']

In [ ]:
df['Y_train']=label_encoder.fit_transform(df['Sentiment'])
df['Y_train']

In [ ]:
final_word_embedding

In [ ]:
dummy=[[0]*768]
def word_embed(Word_Embeddings,max_length=50):
  word_embed=[]
  for we in Word_Embeddings:
    original_we=we
    if(len(original_we)<max_length):
      for i in range(max_length-len(original_we)):
        original_we=original_we+dummy
    if(len(original_we)>max_length):
      original_we=original_we[:max_length]
    word_embed.append(original_we)
  return word_embed
Word_Embeddings=word_embed(final_word_embedding,128)
np.save('Malay_bert_word_embedding_bahasa.npy',Word_Embeddings)

In [ ]:
Word_Embeddings

In [ ]:
# split dataset into test set, train set and unlabel pool
# def split(dataset,labels ,train_size, test_size):
#     x = dataset
#     y = labels
#     x_train, x_pool, y_train, y_pool = train_test_split(
#         x, y, train_size = train_size)
#     unlabel, x_test, label, y_test = train_test_split(
#         x_pool, y_pool, test_size = test_size)
#     return x_train, y_train, x_test, y_test, unlabel, label
# using SVM

In [ ]:
from sklearn.svm import SVC
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from imblearn.over_sampling import ADASYN
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# svclassifier = SVC(kernel='linear',probability=True)
# svclassifier.fit(x_train, y_train)
# prediction_svm = svclassifier.predict(x_test)
# accuracy_svm=sum([1 for i in range(len(x_test)) if y_test[i]==prediction_svm[i]])/len(x_test)
# print(accuracy_svm)
# print(classification_report(y_test,prediction_svm))

In [ ]:
f=open('actv_lr_classification_report.txt','w')
from imblearn.over_sampling import RandomOverSampler
sample = RandomOverSampler(sampling_strategy=0.9)
word_embedding=np.load('Malay_xlnet_word_embedding_bahasa.npy')
sent_embed=np.amax(word_embedding,axis=1)

FileNotFoundError: ignored

In [ ]:
# sent_embed=sent_embed / np.sqrt(np.sum(sent_embed**2))
dataset=np.array(sent_embed)
df=pd.read_csv('hate_data.csv')
arr=df.hate.to_list()
for i in range(len(arr)):
    if arr[i]=='no':
        arr[i]=0
    else:
        arr[i]=1
labels=arr
X, y = sample.fit_resample(dataset, arr)

classifier1 = SVC(kernel='linear',probability=True)
classifier2 = SVC(kernel='linear',probability=True)
ac1, ac2 = [], []

In [ ]:
# read dataset
# for _ in range(10):
# word_embedding=np.load('Malay_xlnet_word_embedding_bahasa.npy')
# sent_embed=np.amax(word_embedding,axis=1)
# data=np.array(sent_embed)

In [ ]:
word_embedding=np.load('Malay_actv_lr_xlnet_word_embedding_bahasa.npy')
sent_embed=np.amax(word_embedding,axis=1)
# sent_embed=sent_embed / np.sqrt(np.sum(sent_embed**2))
actv_lr=np.array(sent_embed)
id_actv_lr=[i for i in range(len(actv_lr))]

actv_lr_csv=pd.read_csv('Malay_actv_lr_data.csv')

In [ ]:
# imputing missing data
#     imputer = SimpleImputer(missing_values = 0, strategy ="mean")
#     imputer = imputer.fit(dataset[:, :-1])
#     dataset[:, :-1] = imputer.transform(dataset[:, :-1])
# labels=np.load('Malay_labels.npy')
# for i in range(len(labels)):
#     if labels[i]==1 or labels[i]==3:
#         labels[i]=1
#     else:
#         labels[i]=0

In [ ]:
print('Initial state : labelled data size-',len(dataset),'unlabelled data size-',len(actv_lr))
print('Initial state : labelled data size-',len(dataset),'unlabelled data size-',len(actv_lr),file=f)

In [ ]:
# labels=np.concatenate((arr,np.array([0]*len(sent_embed))),axis=0)
#     labels=arr
# feature scaling
#     sc = StandardScaler()
#     dataset = sc.fit_transform(dataset)

# run both models 100 times and take the average of their accuracy
 # arrays to store accuracy of different models

In [ ]:
ind=[]
# x_train, x_test, y_train, y_test= train_test_split(X,y ,test_size=0.20)
f1=[]
df=pd.read_csv('Malay_actv_lr_data.csv')
k=0
x_train, X_test, y_train, Y_test= train_test_split(
        X,y ,test_size=0.20)
d={0:"no",1:"yes"}
quant=pd.DataFrame(columns=['labelled','unlabelled','hate_added','non_hate_added','f1_hate','f1_non_hate'])

In [ ]:
dinc=len(dataset)
while(len(actv_lr)>10 and k<500):
#     classifier1 = SVC(kernel='linear',probability=True)
    print('Iteration : ',k)
    print('Iteration : ',k,file=f)
    k=k+1
#     X=data
#     y=labels
    # split dataset into train(5 %), test(25 %), unlabel(70 %)
    x_train, x_test, y_train, y_test= train_test_split(X,y ,test_size=0.10)
    classifier1.fit(x_train, y_train)

In [ ]:
    # train model by active learning
#     for i in range(5):
#     X=x_train
#     y=y_train
#     ada = ADASYN(random_state=42)
#     ada.fit(X, y)
#     x_train, y_train = ada.fit_resample(X, y)

In [ ]:
    pr=classifier1.predict(X_test)
    pr=[d[i] for i in pr]
    l=[d[i] for i in Y_test]
    print(classification_report(l,pr))
    print(classification_report(l,pr),file=f)
    ac1.append(classifier1.score(X_test, Y_test))
    prob=classifier1.predict_proba(actv_lr)
    y_probab = np.array([max(pr) for pr in prob])
    p = 0.75
    uncrt_pt_ind = []
    crt_pt_ind=[]
    di=[]

In [ ]:
    for i in range(actv_lr.shape[0]):
        if(y_probab[i] < p ):
            uncrt_pt_ind.append(i)
        else:
            crt_pt_ind.append(i)
#             print(prob[i])
            actv_lr_csv.loc[id_actv_lr[i],'hate']=d[np.argmax(prob[i])]
            y=np.append([np.argmax(prob[i])],y,axis=0)
            di.append(d[np.argmax(prob[i])])

In [ ]:
#     x_train = np.append(unlabel[uncrt_pt_ind, :], x_train, axis = 0)
#     y_train = np.append(label[uncrt_pt_ind], y_train)
#     unlabel = np.delete(unlabel, uncrt_pt_ind, axis = 0)
#     label = np.delete(label, uncrt_pt_ind)

In [ ]:
    X=np.append(actv_lr[crt_pt_ind,:],X,axis=0)
    dummy_df=df.iloc[crt_pt_ind,:]
    dummy_df['hate']=di
    dummy_df.to_csv('Active_lr/Iteration-'+str(k)+'.csv',index=False)

In [ ]:
#     print(uncrt_pt_ind)
    actv_lr=np.delete(actv_lr,crt_pt_ind,axis=0)
    id_actv_lr=np.delete(id_actv_lr,crt_pt_ind,axis=0)
#     classifier2.fit(x_train, y_train)
#     ac1.append(classifier2.score(x_test, y_test))

In [ ]:
    dinc+=di.count('no')+di.count('yes')
    print('labelled data size : ',dinc)
    print('unlabelled data size : ',len(actv_lr))
    print('NON HATE examples added : ', di.count('no'))
    print('HATE examples added : ',di.count('yes'))


In [ ]:
    print('labelled data size : ',dinc,file=f)
    print('unlabelled data size : ',len(actv_lr),file=f)
    print('NON HATE examples added : ', di.count('no'),file=f)
    print('HATE examples added : ',di.count('yes'),file=f)

In [ ]:
    f1=f1_score(l,pr,average=None)
    quant.loc[len(quant)]=[len(x_train),len(actv_lr),di.count('yes'),di.count('no'),list(f1)[1],list(f1)[0]]
    ''' split dataset into train(same as generated by our model),
    test(25 %), unlabel(rest) '''

In [ ]:
#     train_size = x_train.shape[0]/dataset.shape[0]
#     x_train, y_train, x_test, y_test, unlabel, label = split(
#         dataset,labels ,train_size, 0.20)

    # train model without active learning
#     classifier3 = SVC(kernel='linear',probability=True)
#     classifier3.fit(x_train, y_train)
#     ac2.append(classifier3.score(x_test, y_test))

In [ ]:
print("Accuracy by active model :", np.mean(ac1)*100)
print("Accuracy by active model :", np.mean(ac1)*100,file=f)
quant.to_csv('actv_lr_data.csv',index=False)
actv_lr_csv.to_csv('Malay_actv_lr_data_test.csv',index=False)
# print("Accuracy by random sampling :", mean(ac2)*100)